# IA353 - Exercícios de Fixação de Conceitos 

Aluno: Gabriel Ayres de Oliveira

Data: 31/03/2021

In [ ]:
# Carregando módulos utilizados no trabalho e variáveis comuns
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

ra = 234619

## Questão 1: 

### Classificação de Dígitos usando a base MNIST

Para esse exercício treina-se a rede neural usando a forma fechada. Porém visto que ele é um problema sobredeterminado, deve-se ajustar coeficientes para ter uma solução melhor.

In [ ]:
def visualizaDigito(matrizDados, indice, tamanho):
    
    #Reorganizando o vetor da imagem em uma matriz
    imagem = np.reshape(matrizDados[indice], [tamanho, tamanho])
    
    #Corrigindo sua orientação
    imagem = imagem.T
    
    return imagem

In [ ]:
def calculaPesos(setEntrada, setSaida, coeficiente):
    
    #Calculando os pesos de forma fechada
    a = np.matmul(setEntrada.T, setEntrada) #X.T * X
    diagonal = np.eye(a.shape[0], dtype = int) #I
    b = np.multiply(diagonal,coeficiente) #lambda * I
    pesos = np.matmul(np.matmul(np.linalg.inv(a + b), setEntrada.T), setSaida)  

    return pesos

In [ ]:
def calculaTaxa(saidaCalculada, saidaOriginal):
    acerto = 0
    for i in range(saidaOriginal.shape[0]):
        if (np.argmax(saidaCalculada[i,:]) == np.argmax(saidaOriginal[i,:])):
            acerto = acerto + 1
    
    taxaAcerto = acerto/saidaCalculada.shape[0]
    
    return taxaAcerto

In [ ]:
#Carregando os dados MNIST
data = scipy.io.loadmat("data.mat")
test = scipy.io.loadmat("test.mat")

Xtr = data["X"]
Str = data["S"]

Xts = test["Xt"]
Sts = test["St"]

#Reorganizando os dados de acordo com o RA do aluno
np.random.seed(ra)

permutacao = np.random.permutation(Xtr.shape[0])
Xtr[:] = Xtr[permutacao] 
Str[:] = Str[permutacao]

#Separando o conjunto de treino em dois conjuntos, treino e validação, da forma 80/20
tamanhoValidacao = int(0.2 * Xtr.shape[0])
tamanhoTreino = Xtr.shape[0] - tamanhoValidacao

Xvl = Xtr[:tamanhoValidacao]
Svl = Str[:tamanhoValidacao]

Xtr = Xtr[tamanhoValidacao:]
Str = Str[tamanhoValidacao:]

plt.figure()
plt.title("Dígito")
plt.imshow(visualizaDigito(Xvl,200,28))
plt.axis("off")
plt.show()

In [ ]:
#Calculando o coeficiente que implica no menor erro de classificaćão
intervaloPesquisa = np.arange(-10,16,2)
intervaloFino = intervaloPesquisa.shape[0]
k=0;

#Cálculo do coeficiente grosseiro
vetorErroGrosseiro = np.zeros_like(intervaloPesquisa, dtype = float)
for i in intervaloPesquisa:
    pesos = calculaPesos(Xtr, Str, np.power(2.,i))
    Scalc = np.matmul(Xvl,pesos)
    vetorErroGrosseiro[k] = calculaTaxa(Scalc, Svl)
    k = k + 1
    print(k)

indiceMenorErro = np.argmax(vetorErroGrosseiro)
    

#Cálculo do coeficiente fino
intervaloPesquisaFina = np.linspace(intervaloPesquisa[indiceMenorErro-1],intervaloPesquisa[indiceMenorErro+1], intervaloFino)
vetorErroFino = np.zeros_like(intervaloPesquisaFina, dtype = float)
k = 0

for i in intervaloPesquisaFina:
    pesos = calculaPesos(Xtr, Str, np.power(2.,i))
    Scalc = np.matmul(Xvl,pesos)
    vetorErroFino[k] = calculaTaxa(Scalc, Svl)
    k = k + 1

In [ ]:
#Recalculando para o menor coeficiente de erro
print("Usando o menor valor de coeficiente calculado: " + str(np.max(vetorErroFino)))
pesos = calculaPesos(Xtr, Str, np.power(2.,intervaloPesquisaFina[np.argmax(vetorErroFino)]))

In [ ]:
#Plotando ambos os erros num semilog
plt.figure()

plt.subplot(2,1,1)
plt.semilogx(intervaloPesquisa, vetorErroGrosseiro)

plt.subplot(2,1,2)
plt.semilogx(intervaloPesquisaFina,vetorErroFino)

plt.show()

In [ ]:
#Imprimindo os dígitos 

plt.figure(figsize=(20,10))

for i in range (0,10):
    
    plt.subplot(2 ,5, i+1)
    plt.title("Dígito " + str(i+1))
    plt.imshow(pesos[:,i].reshape([28,28]).T)

plt.show()


